In [1]:
import numpy as np
import random
import networkx as nx

In [2]:
G1 = nx.read_adjlist('datasets/MinCut.txt', nodetype=int) #create_using=nx.MultiGraph()   Now, nx.Graph() is default

In [3]:
len(G1.edges())

2517

In [4]:
len(G1.nodes())

200

In [5]:
!cat datasets/cut_test.txt

1 2 3 4 7
2 1 3 4
3 1 2 4
4 1 2 3 5
5 4 6 7 8
6 5 7 8
7 1 5 6 8
8 5 6 7

In [6]:
G_t = nx.read_adjlist('datasets/cut_test.txt', nodetype=int)

In [7]:
G_t.nodes()

[1, 2, 3, 4, 7, 5, 6, 8]

In [8]:
G_t.edges()

[(1, 2),
 (1, 3),
 (1, 4),
 (1, 7),
 (2, 3),
 (2, 4),
 (3, 4),
 (4, 5),
 (7, 5),
 (7, 6),
 (7, 8),
 (5, 6),
 (5, 8),
 (6, 8)]

In [9]:
len(G_t.edges())

14

In [13]:
# Below is my way, far from ideal
# Karger´s min cut algorithm

def rc(G):
    n = len(G.nodes())
    N = int(n**2 * np.log(n))
    min_num_edges = None
    
    for i in range(200):  # for N is too slow
        nodes = G.nodes()
        edges = G.edges()
        while len(nodes) > 2:
            r = random.randint(0,len(edges)-1)
            cut_edge = edges.pop(r)
            cut_node = cut_edge[1]
            kept_node = cut_edge[0]
            nodes = [x for x in nodes if x != cut_node]
            temp = []
            for edge in edges:
                if edge[0] != cut_node and edge[1] != cut_node:
                    temp.append(edge)
                
                elif edge[0] == cut_node and edge[1] != kept_node:
                    temp.append((kept_node, edge[1]))
                    
                elif edge[1] == cut_node and edge[0] != kept_node:
                    temp.append((kept_node, edge[0]))
                    
            edges = temp
            
        num_edges = len(edges)
        if min_num_edges == None or min_num_edges > num_edges:
            min_num_edges = num_edges
    
    return min_num_edges

In [14]:
rc(G_t)

2

In [16]:
rc(G1)

17

In [ ]:
# 2018-10-13

<img src='images/min_cut1.png' style="width:650px">

<img src='images/min_cut2.png' style="width:650px">

<img src='images/min_cut3.png' style="width:650px">

In [10]:
import random
import re
import copy


def load_graph(file_path):
    with open(file_path) as f:
        lines = f.readlines()
    lines = map(lambda s: re.sub('\s+', ' ', s).strip(), lines)
    lines = map(lambda s: s.split(' '), lines)
    g = {}
    for line in lines:
        g[int(line[0])] = list(map(lambda s: int(s), line[1:])) 

    return g

def get_nodes(graph):
    return [v for v in graph.keys()]

def get_edges(graph):
    edges_list = []
    for v1 in graph.keys():
        for v2 in graph[v1]:
            edges_list.append((v1, v2))
    return edges_list


def contract_edges(g):
    nodes = get_nodes(g)
    num_of_nodes = len(nodes)
    assert num_of_nodes >= 2, "Stop contraction nodes less than or equal to 2"
    # get a random edge
    v1 = nodes[random.randint(0, num_of_nodes - 1)]
    k = random.randint(0, len(g[v1]) - 1)
    v2 = g[v1].pop(k)
    # extend v1 edges and remove v2 from g
    g[v1].extend(g[v2])
    del g[v2]
    # replace occurence of v2 with v1
    for p, q in g.items():
        g[p] = [v1 if v == v2 else v for v in g[p]]
    # remove v1 from itself
    g[v1] = [v for v in g[v1] if v != v1]
    return g


def num_of_cuts(g):
    while len(get_nodes(g)) > 2:
        contract_edges(g)
    key = list(g.keys())[0]
    return len(g[key])


def min_cuts(g):
    min_ = 9999999
    for _ in range(0, 10):
        new_g = copy.deepcopy(g)
        cuts = num_of_cuts(new_g)
        if cuts < min_:
            min_ = cuts
    return min_

In [11]:
test_graph = load_graph('datasets/cut_test.txt')

In [12]:
min_cuts(test_graph)

2

In [13]:
my_graph = load_graph('datasets/MinCut.txt')

In [14]:
min_cuts(my_graph)

17